In [26]:
import torch
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn, optim

In [27]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [28]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

In [29]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)
bobs_data

(Wrapper)>[PointerTensor | me:26865179346 -> bob:58482532225]

In [23]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.2905) Alice:tensor(2.3935)
Bob:tensor(0.2318) Alice:tensor(0.3573)
Bob:tensor(0.1941) Alice:tensor(0.2848)
Bob:tensor(0.1648) Alice:tensor(0.2369)
Bob:tensor(0.1404) Alice:tensor(0.1972)
Bob:tensor(0.1197) Alice:tensor(0.1641)
Bob:tensor(0.1021) Alice:tensor(0.1366)
Bob:tensor(0.0871) Alice:tensor(0.1137)
Bob:tensor(0.0743) Alice:tensor(0.0946)
Bob:tensor(0.0634) Alice:tensor(0.0788)


In [15]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [16]:
with torch.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

In [17]:
# Rinse and Repeat
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0018) Alice:tensor(0.0295)
Bob:tensor(0.0005) Alice:tensor(0.0145)
Bob:tensor(0.0009) Alice:tensor(0.0069)
Bob:tensor(0.0014) Alice:tensor(0.0033)
Bob:tensor(0.0018) Alice:tensor(0.0016)
Bob:tensor(0.0018) Alice:tensor(0.0008)
Bob:tensor(0.0017) Alice:tensor(0.0004)
Bob:tensor(0.0015) Alice:tensor(0.0002)
Bob:tensor(0.0013) Alice:tensor(0.0001)
Bob:tensor(0.0011) Alice:tensor(7.8389e-05)


In [19]:
preds = model(data)
loss = ((preds - target) ** 2).sum()
print(preds)
print(target)
print(loss.data)

tensor([[0.0994],
        [0.0853],
        [0.8916],
        [0.8774]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0439)
